This notebook is intended to be used in a <strong>SageMaker</strong> notebook

First, select an environment with <strong>python 3.7</strong>

In [ ]:
import os
CWD = os.getcwd()
CWD

In [ ]:
!git clone --depth 1 https://github.com/heartexlabs/label-studio.git {CWD}/label-studio

In [ ]:
!rm -rf {CWD}/label-studio

In [ ]:
!which label-studio

## Install

In [ ]:
%%capture
%pip install ruamel.yaml --upgrade

In [ ]:
%%capture
%pip install -e {CWD}/label-studio

#### Fix values

In [ ]:
old_value = "os.path.join(BASE_DIR, 'static_build')"
new_value = "os.path.join(BASE_DIR, 'static')"
!sed -i "s+{old_value}+{new_value}+g" {CWD}/label-studio/label_studio/core/settings/base.py

old_value = "STATICFILES_DIRS"
new_value = "#STATICFILES_DIRS"
!sed -i "s+{old_value}+{new_value}+g" {CWD}/label-studio/label_studio/core/settings/base.py

old_value = "STATICFILES_STORAGE"
new_value = "#STATICFILES_STORAGE"
!sed -i "s+{old_value}+{new_value}+g" {CWD}/label-studio/label_studio/core/settings/base.py

old_value = "USE_ENFORCE_CSRF_CHECKS = "
new_value = "USE_ENFORCE_CSRF_CHECKS = False  #"
!sed -i "s+{old_value}+{new_value}+g" {CWD}/label-studio/label_studio/core/settings/base.py

## Configuration

In [ ]:
import json
log_path = '/opt/ml/metadata/resource-metadata.json'
with open(log_path, 'r') as logs:
    _logs = json.load(logs)
INSTANCE_NAME = _logs['ResourceName']
URL_NOTEBOOK = f"https://{INSTANCE_NAME}.notebook.eu-central-1.sagemaker.aws"
DOCUMENT_ROOT = "/tmp/bucket/"
DATABASE_NAME = f"{CWD}/db.sqlite"

In [ ]:
%env LABEL_STUDIO_PORT=6006
%env LABEL_STUDIO_HOST={URL_NOTEBOOK}/proxy/6006/
%env LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED=true
%env LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT={DOCUMENT_ROOT}
%env LABEL_STUDIO_DATABASE_NAME={DATABASE_NAME}

## Start

In [ ]:
%%capture
!label-studio start --username email@email.com --password password

## Get token

In [ ]:
!label-studio user --username email@email.com

## Alternative mode for starting the server (useful to export the annotations)

In [ ]:
!python ./label_studio/manage.py runserver localhost:6006

In [ ]:
!python label_studio/manage.py collectstatic --no-input

In [ ]:
!git clone https://github.com/heartexlabs/label-studio-frontend.git /home/ec2-user/SageMaker/label-studio/label-studio-frontend

In [ ]:
%cd /home/ec2-user/SageMaker/label-studio/label-studio-frontend